Install Ngrok, SQLAlchemy, Flask-Session.

Mount Google Drive for File Access & setup token.

In [ ]:
!pip install pyngrok
!pip install flask_sqlalchemy
!pip install Flask-Session

import os, getpass

from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define where your token lives (or would live)
TOKEN_PATH = "/content/drive/MyDrive/ngrok_token.txt"

# 3. Load or prompt
if os.path.exists(TOKEN_PATH):
    with open(TOKEN_PATH, "r") as f:
        ngrok_token = f.read().strip()
    print("Loaded Ngrok auth token from file.")
else:
    ngrok_token = getpass.getpass("Enter your Ngrok auth token: ")

# 4. Export into env so flask_ngrok can pick it up
os.environ["NGROK_AUTH_TOKEN"] = ngrok_token


Latest Server Code

In [ ]:
# Core Python modules
import os
import sys
import random
import logging
import sqlite3

# Flask and extensions
from flask import Flask, request, jsonify, render_template, session
from flask_session import Session

# Third-party libraries
from pyngrok import ngrok
from werkzeug.security import generate_password_hash, check_password_hash

logging.basicConfig(level=logging.DEBUG)

# Define project paths
BASE       = "/content/drive/MyDrive/colab-web-app"
TEMPLATES  = os.path.join(BASE, "templates")
STATIC     = os.path.join(BASE, "static")
ROUTES_DIR = os.path.join(BASE, "routes")
DB_PATH    = os.path.join(BASE, "database", "site.db")

# Confirm route folder exists
print("ROUTES folder →", ROUTES_DIR)
!ls "{ROUTES_DIR}"

# Ensure Python can import from there
sys.path.insert(0, ROUTES_DIR)

# Create Flask app & register Blueprint
app = Flask(
    __name__,
    template_folder=TEMPLATES,
    static_folder=STATIC
)

# Inject your DB_PATH into app.config **before** registering routes
app.config['DB_PATH'] = DB_PATH

from main_routes import main_bp
app.register_blueprint(main_bp)

from auth_routes import auth_bp
app.register_blueprint(auth_bp)

from admin_routes import admin_bp
app.register_blueprint(admin_bp)

# Session & Secret Key
app.config["SECRET_KEY"]   = "supersecretkey"
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

# Database Connection Function
def get_db_connection():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn

from functools import wraps
from flask import session, redirect, url_for, flash
from flask import jsonify


# Ngrok Authentication & Tunnel Setup
auth_token = os.getenv("NGROK_AUTH_TOKEN")
if auth_token:
    ngrok.set_auth_token(auth_token)

public_url = ngrok.connect(8000)
print(f'Access the web app here: {public_url}')

# Run Flask Server
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)
